In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import yfinance as yf
import seaborn as sns

from arch import arch_model
from sklearn.model_selection import train_test_split
from arch.__future__ import reindexing

from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def add_log_returns(close):
    log_returns = np.log(close/close.shift())
    return log_returns

# def add_GARCH(close, lr, test_size = 1250):
#     rolling_predictions = list()

#     for i in range(test_size):
#         train = lr.dropna()[:-(test_size-i)]
#         model = arch_model(100 * train, p=1, q=1, mean='constant', vol='GARCH')
#         model_fit = model.fit(disp='off')
#         pred = model_fit.forecast(horizon=1, reindex=True)
#         rolling_predictions.append(np.sqrt(pred.variance.values[-1,:][0]))

#     GARCH = pd.Series(rolling_predictions, index=lr.index[-test_size:]) / 100

#     return GARCH

## Volatility spillover analysis

- Variance Decomposition: Variance decomposition techniques, such as the Cholesky decomposition, can help quantify the contribution of each market to the overall volatility spillover. By decomposing the total volatility into individual components, you can assess the extent to which shocks in one market affect the volatility of other markets.

- Granger Causality: Granger causality tests can determine if past values of one market's volatility help predict the future volatility of another market. If there is significant causality between two markets, it suggests the presence of volatility spillover. Various time series econometric methods, such as Vector Autoregression (VAR) or Multivariate GARCH models, can be used to perform Granger causality tests.

- Spillover Index: Spillover indices, such as the Diebold and Yilmaz spillover index, measure the interconnectedness or transmission of volatility between different markets. These indices provide a quantitative measure of the volatility spillover effects across markets, indicating the strength and direction of transmission.

- Co-integration Analysis: Co-integration analysis examines the long-term relationship between multiple markets. If there is a co-integrating relationship among markets, it suggests a long-term equilibrium and the possibility of volatility spillover.

- Correlation and Cross-Correlation Analysis: Correlation analysis can identify the degree of linear relationship between market volatilities. High correlation indicates a potential for volatility spillover. Cross-correlation analysis examines the lagged correlations between market volatilities to determine if there is a lead-lag relationship or spillover effect.

In [28]:
tickers = ['BZ=F', 'GC=F', 'OSEBX.OL', '^GSPC']

log_returns = [ f'lr_{tic}' for tic in tickers]
# Import stock data
data = pd.DataFrame(yf.download(tickers, period='5Y', interval='1d', progress=False, ignore_tz=True)['Adj Close'])

# Forward fill
data = data.ffill()

# Add log_returns and GARCH
for tic in tickers:
    data[f'lr_{tic}'] = add_log_returns(data[tic])
    # data[f'GARCH_{tic}'] = add_GARCH(data[tic], data[f'lr_{tic}'])

display(data.dropna())

,BZ=F,GC=F,OSEBX.OL,^GSPC,lr_BZ=F,lr_GC=F,lr_OSEBX.OL,lr_^GSPC
Date,,,,,,,,
2018-07-03,77.760002,1251.599976,877.979980,2713.219971,-0.020997,0.009473,0.005998,-0.004960
2018-07-04,77.760002,1251.599976,879.429993,2713.219971,0.000000,0.000000,0.001650,0.000000
2018-07-05,77.389999,1257.300049,883.390015,2736.610107,-0.004770,0.004544,0.004493,0.008584
2018-07-06,77.110001,1254.300049,877.340027,2759.820068,-0.003625,-0.002389,-0.006872,0.008446
2018-07-09,78.070000,1258.099976,888.659973,2784.169922,0.012373,0.003025,0.012820,0.008784
...,...,...,...,...,...,...,...,...
2023-06-26,74.180000,1923.699951,1195.699951,4328.819824,0.004459,0.002394,0.001431,-0.004497
2023-06-27,72.260002,1914.000000,1186.150024,4378.410156,-0.026224,-0.005055,-0.008019,0.011391
2023-06-28,74.029999,1912.300049,1198.670044,4376.859863,0.024200,-0.000889,0.010500,-0.000354


### Vector Autoregression

In [83]:

var_data = data[log_returns].dropna()

test_prop = 0.25
test_size = (int(len(var_data)*test_prop))

cointegration_results = coint_johansen(var_data, det_order=0, k_ar_diff=1)
eigenvalues = cointegration_results.eig

optimal_lags = np.argmax(eigenvalues)


train = var_data.iloc[:-test_size]
test = var_data.iloc[-test_size:]

model = VAR(train)
results = model.fit(optimal_lags)

lag_order = results.k_ar

display(results.summary())

forecast = results.forecast(train.values[-lag_order:], steps=len(test))

# Convert the forecasted values to a DataFrame
forecast_df = pd.DataFrame(forecast, index=test.index, columns=test.columns)

# Evaluate the forecast accuracy
mse = ((forecast_df - test) ** 2).mean()
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

c:\Users\Jone\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 05, Jul, 2023
Time:                     00:21:26
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                   -34.2007
Nobs:                     966.000    HQIC:                  -34.2132
Log likelihood:           11049.9    FPE:                1.37427e-15
AIC:                     -34.2209    Det(Omega_mle):     1.36860e-15
--------------------------------------------------------------------
Results for equation lr_BZ=F
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.000315         0.000986            0.320           0.749

Results for equation lr_GC=F
           coefficient       std. error           t-stat            prob
------------------------------------------------------------

IndexError: index 0 is out of bounds for axis 0 with size 0